In [1]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.trotterisation import make_trotterisation_routine
from qat.fermion.chemistry.ucc import construct_ucc_ansatz, guess_init_params, get_hf_ket, get_cluster_ops
from qat.qpus import get_default_qpu

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

In [2]:
import numpy as np
from qat.core import Observable, Term
import numpy as np
import scipy
from numpy import binary_repr
from qat.fermion import ElectronicStructureHamiltonian
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation

from qat.lang.AQASM import Program, QRoutine, RY, CNOT, RX, Z, H, RZ, I, X
from qat.core import Observable, Term, Circuit
from qat.lang.AQASM.gates import Gate
import matplotlib as mpl
import numpy as np
from typing import Optional, List
import warnings
from qat.qpus import get_default_qpu

qpu = get_default_qpu()
method = "BFGS"


In [3]:
import numpy as np
import scipy
from numpy import binary_repr
from qat.fermion import ElectronicStructureHamiltonian
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry.ucc import (
    convert_to_h_integrals,
    transform_integrals_to_new_basis,
)
from qat.fermion.chemistry.ucc_deprecated import (
    get_active_space_hamiltonian,
    get_cluster_ops_and_init_guess,
)
from qat.fermion.transforms import (
    get_bk_code,
    get_jw_code,
    get_parity_code,
    recode_integer,
    transform_to_bk_basis,
    transform_to_jw_basis,
    transform_to_parity_basis,
)

from openvqe.common_files.generator_excitations import (
    singlet_gsd,
    singlet_sd,
    singlet_upccgsd,
    spin_complement_gsd,
    spin_complement_gsd_twin,
    uccsd,
)

In [4]:
import numpy as np
from itertools import product
from openvqe.common_files.fermion_util import order_fermionic_term

import scipy.optimize
from qat.fermion.chemistry.ucc_deprecated import build_ucc_ansatz
from qat.lang.AQASM import Program
from qat.qpus import get_default_qpu


def _apply_transforms(cluster_ops_fr, transform, perm=0):
    if transform == "JW":
        transform_func = transform_to_jw_basis
    elif transform == "Bravyi-Kitaev":
        transform_func = transform_to_bk_basis
    elif transform == "parity_basis":
        transform_func = transform_to_parity_basis
    else:
        return
    
    cluster_ops = []
    cluster_ops_sp = []
    for y in cluster_ops_fr:
        hamilt_sp = transform_func(y)
        if hamilt_sp.terms != []:
            cluster_ops.append(y)
            cluster_ops_sp.append(hamilt_sp)
    cluster_ops += cluster_ops * perm
    cluster_ops_sp += cluster_ops_sp * perm
    pool_size = len(cluster_ops_sp)
    return pool_size, cluster_ops, cluster_ops_sp
from qat.core import Term
#from qat.dqs import FermionHamiltonian
from qat.fermion import FermionHamiltonian as Hamiltonian

import numpy as np
from itertools import product
import itertools
from itertools import combinations

from qat.core import Term
#from qat.dqs import FermionHamiltonian
from qat.fermion import FermionHamiltonian as Hamiltonian



def merge_duplicate_terms(hamiltonian):

    """
    Take a fermionic Hamiltonian and merge terms with same operator content

    Parameters
    ----------
    hamiltonian : Hamiltonian
        Of type fermionic cluster operator

    Returns
    ----------
    merged_hamiltonian: Hamiltonian
        The listed merged operators
    
    """
    

    terms = {}

    for term in hamiltonian.terms:

        key = tuple([term.op, tuple(term.qbits)])

        if key in terms.keys():

            terms[key] += term.coeff

        else:

            terms[key] = term.coeff

    terms = [Term(v, k[0], list(k[1])) for k, v in terms.items()]

    merged_hamiltonian = Hamiltonian(
        hamiltonian.nbqbits, terms=terms, constant_coeff=hamiltonian.constant_coeff
    )

    return merged_hamiltonian

import numpy as np
from itertools import product

from qat.core import Term
#from qat.dqs import FermionHamiltonian
from qat.fermion import FermionHamiltonian as Hamiltonian

def permute_fermionic_operator(fermionic_term, ind):
    """
    Perform the permutation of the two operators in index ind and ind + 1 in a fermionic Term pauli string

    Args:
        fermionic_term (Term): the fermionic term which operators we seek to permute
        ind (int): the lower index of the two consecutive creation or annihilation operators we seek to permute
    
    Returns:
        list_terms (list<Term>): the list of fermionic terms resulting of the permutation
    """
    coeff = fermionic_term.coeff
    pauli_op = fermionic_term.op
    qbits = fermionic_term.qbits

    if ind >= len(pauli_op) - 1:
        raise IndexError
    permuted_pauli_op = pauli_op[:ind] + pauli_op[ind + 1] + pauli_op[ind] + pauli_op[ind + 2:]
    permuted_qbits = qbits[:]
    permuted_qbits[ind], permuted_qbits[ind + 1] = permuted_qbits[ind + 1], permuted_qbits[ind]
    if 'c' in pauli_op[ind : ind + 2] and 'C' in pauli_op[ind : ind + 2] and qbits[ind] == qbits[ind + 1]:
        return [Term(coefficient=coeff, pauli_op=pauli_op[: ind] + pauli_op[ind + 2:], qbits=qbits[:ind] + qbits[ind + 2:]), 
                Term(coefficient=-coeff, pauli_op=permuted_pauli_op, qbits=permuted_qbits)]
    else:
        return [Term(coefficient=-coeff ,pauli_op=permuted_pauli_op , qbits=permuted_qbits)]

def order_qubits(fermionic_term):
    """
    Takes a fermionic term which pauli_op is supposed to be ordered properly, and reorder it increasing qbit numbers

    Args:
        fermionic_term (Term): the term to reorder

    Returns:
        ordered_term (Term): the reordered term
    """
    coeff = fermionic_term.coeff
    pauli_op = fermionic_term.op
    qbits = fermionic_term.qbits

    ind_c = pauli_op.index("c")
    qbits_C = qbits[:ind_c]
    qbits_c = qbits[ind_c:]
    new_qbits = []
    
    for qbits_op in [qbits_C, qbits_c]:
        qbits_temp = qbits_op[:]
        ordered = False
        while not ordered:
            ind = 0
            while ind < len(qbits_temp) - 1 and qbits_temp[ind] <= qbits_temp[ind + 1]:
                if qbits_temp[ind] == qbits_temp[ind + 1]:
                    return 
                ind += 1
            if ind < len(qbits_temp) - 1:
                ind += 1
                new_ind = 0
                while qbits_temp[new_ind] < qbits_temp[ind]:
                    new_ind += 1
                elt_not_in_order = qbits_temp.pop(ind)
                qbits_temp.insert(new_ind, elt_not_in_order)
                coeff *= (-1)**(ind - new_ind)
            else:
                ordered = True
        new_qbits += qbits_temp
    return Term(coefficient=coeff, pauli_op=pauli_op, qbits=new_qbits)

def order_fermionic_ops(fermionic_term):
    """
    Order the operators list of a fermionic_term by putting the creations operators on the left and the annihilation operators on the right, with respect to the fermionic anticommutation relations.

    Args:
         fermionic_term (Term): the term to order

    Returns:
        ordered_fermionic_terms (list<Term>): the list of ordered fermionic terms
    """
    coeff = fermionic_term.coeff
    pauli_op = fermionic_term.op
    qbits = fermionic_term.qbits

    ind_c = pauli_op.index('c')
    try:
        ind_C = pauli_op[ind_c:].index('C') + ind_c
    except ValueError:
        new_terms = [fermionic_term]
        ordered_pauli_op = True
    else:
        new_terms = []
        for new_fermionic_term in permute_fermionic_operator(fermionic_term, ind_C - 1):
            new_terms += order_fermionic_term(new_fermionic_term)
    return new_terms

def order_fermionic_term(fermionic_term):
    """
    Order any fermionic term by putting the creation operators on the left, ordered by increasing qubit numbers, and the annihilation operators on the right, ordered y increasing qubit numbers, with respect to the fermionic anticommutation relations.

    Args:
        fermionic_term (Term): the term to order

    Returns:
        ordered_fermionic_terms (list<Term>): the list of ordered fermionic terms
    """
    new_terms = order_fermionic_ops(fermionic_term)
    ordered_terms = []
    for new_term in new_terms:
        ordered_term = order_qubits(new_term)
        if ordered_term:
            ordered_terms.append(ordered_term)
    return ordered_terms

#-----------------------------------------------------------UCCGSD_MAN-----------------------------------------------------------------



from typing import List, Tuple
from qat.core import Term
from qat.fermion import FermionHamiltonian as Hamiltonian

def uccgsd_man(n_elec, orbital_number, transform,
               single_terms: List[List[Tuple[int, str, List[int]]]],
               double_terms: List[List[Tuple[int, str, List[int]]]]):
    """
    Generate fermionic and spin cluster operators for UCC-GSD ansatz.

    Parameters:
    n_elec (int): Number of electrons.
    orbital_number (int): Number of orbitals.
    transform (str): Type of transformation. Either 'JW', 'Bravyi-Kitaev', or 'parity_basis'.
    single_terms (List[List[Tuple[int, str, List[int]]]]): List of single terms for spin complement.
    double_terms (List[List[Tuple[int, str, List[int]]]]): List of double terms for spin complement.

    Returns:
    tuple: (pool_size, cluster_ops, cluster_ops_sp)
    """
    spin_complement_single = []
    spin_complement_double = []

    # Create single terms
    for single_term in single_terms:
        term_objects = [Term(coeff, op, qbits) for coeff, op, qbits in single_term]
        hamiltonian_single = Hamiltonian(2 * orbital_number, term_objects)
        spin_complement_single.append(hamiltonian_single)

    # Create double terms
    for double_term in double_terms:
        term_objects = [Term(coeff, op, qbits) for coeff, op, qbits in double_term]
        hamiltonian_double = Hamiltonian(2 * orbital_number, term_objects)
        spin_complement_double.append(hamiltonian_double)

    spin_complements = spin_complement_single + spin_complement_double
    return _apply_transforms(spin_complements, transform)


#-----------------------------------------------------------UCCGSD-----------------------------------------------------------------



def uccgsd(n_elec, orbital_number, transform):
    """
    TBD

    Parameters
    ----------
    n_elec: int
        The number of electrons

    orbital_number: int
        The number of orbitals
    
    transform: string
        type of transformation. Either 'JW', 'Bravyi-Kitaev', or 'parity_basis'
    
    Returns
    -------

    pool_size: int
        The number of the cluster operators

    cluster_ops: List<Hamiltonian>
        List of fermionic cluster operators
        
    cluster_ops_sp: List<Hamiltonian>
        List of spin cluster operators

    """
    spin_complement_single = []
    spin_complement_double = []

    for p in range(0, 2*orbital_number):
        for q in range(p, 2*orbital_number):
            term_a = [
                    Term(1, "Cc", [p, q]),
                    Term(-1, "Cc", [q, p]),
                ]
            hamiltonian = Hamiltonian(2*orbital_number, term_a)
            spin_complement_single.append(hamiltonian)

            for r in range(p, 2*orbital_number):
                for s in range(q if r==p else r, 2*orbital_number):

                    term_a = [
                        Term(1, "CCcc", [p, q, r, s]),
                        Term(-1, "CCcc", [s, r, q, p]),
                        
                    ]
                    
                    term_a_ordered_terms = map(order_fermionic_term, term_a)
                    term_a = sum(term_a_ordered_terms, [])
                    hamiltonian = Hamiltonian(2*orbital_number, terms=term_a)
                    spin_complement_double.append(hamiltonian)

    spin_complements = spin_complement_single + spin_complement_double
    return _apply_transforms(spin_complements, transform)


#-----------------------------------------------------------UCC-action--------------------------------------------------------------


def ucc_action(theta_current, hamiltonian_sp, cluster_ops_sp, hf_init_sp):
        """
        It maps the exponential of cluster operators ("cluster_ops_sp") associated by their parameters ("theta_current")
        using the CNOTS-staircase method, which is done by "build_ucc_ansatz" which creates the circuit on the top of
        the HF-state ("hf_init_sp"). Then, this function also calculates the expected value of the hamiltonian ("hamiltonian_sp").

        Parameters
        ----------
        theta_current: List<float>
            the Parameters of the cluster operators
        
        hamiltonian_sp: Hamiltonian
                Hamiltonian in the spin representation
            
        cluster_ops_sp: list[Hamiltonian]
            list of spin cluster operators
        
        hf_init_sp: int
            the integer corresponds to the hf_init (The Hartree-Fock state in integer representation) obtained by using
            "qat.fermion.transforms.record_integer".
        
        Returns
        --------
            res.value: float
                the resulted energy

        """
        qpu = 0
        prog = 0
        reg = 0
        qpu = get_default_qpu()
        prog = Program()
        reg = prog.qalloc(hamiltonian_sp.nbqbits)
        qrout = 0
        for n_term, (term, theta_term) in enumerate(zip(cluster_ops_sp, theta_current)):
            init = hf_init_sp if n_term == 0 else 0
            qprog = build_ucc_ansatz([term], init, n_steps=1)
            prog.apply(qprog([theta_term]), reg)
        circ = prog.to_circ()
        curr_state =circ
        return curr_state

def ucc_action_energy(hamiltonian_sp, cluster_ops_sp, hf_init_sp, theta_current):
    """
    It maps the exponential of cluster operators ("cluster_ops_sp") associated by their parameters ("theta_current")
    using the CNOTS-staircase method, which is done by "build_ucc_ansatz" which creates the circuit on the top of
    the HF-state ("hf_init_sp"). Then, this function also calculates the expected value of the hamiltonian ("hamiltonian_sp").

    Parameters
    ----------
    hamiltonian_sp: Hamiltonian
        Hamiltonian in the spin representation

    cluster_ops_sp: list[Hamiltonian]
        list of spin cluster operators
    
    hf_init_sp: int
        the integer corresponds to the hf_init (The Hartree-Fock state in integer representation) obtained by using
        "qat.fermion.transforms.record_integer".
    
    theta_current: List<float>
        the Parameters of the cluster operators
    
    Returns
    --------
        res.value: float
            the resulted energy

    """
    qpu = get_default_qpu()
    prog = Program()
    reg = prog.qalloc(hamiltonian_sp.nbqbits)
    for n_term, (term, theta_term) in enumerate(zip(cluster_ops_sp, theta_current)):
        init = hf_init_sp if n_term == 0 else 0
        qprog = build_ucc_ansatz([term], init, n_steps=1)
        prog.apply(qprog([theta_term]), reg)
    circ = prog.to_circ()
    res = qpu.submit(circ.to_job(job_type="OBS", observable=hamiltonian_sp))
    return res.value

#-----------------------------------------------------------ADAPT_STATE--------------------------------------------------------------


def prepare_adapt_state(reference_ket, spmat_ops, parameters):
    """
    Computes the action of the matrix exponential of fermionic sparse operators ("spmat_ops") on
    the state initiated by "reference_ket".

    Parameters
    -----------
    reference_ket: ndarray
        the initial state
    
    spmat_ops: List<transposable linear operator>
        the sparse fermionic operators

    parameters: List<float>
        list of parameters for the "spmat_ops"
    

    Returns
    --------
    new_state: ndarray
        the new state

    """
    new_state = reference_ket * 1.0
    for k in range(len(parameters)):
        new_state = scipy.sparse.linalg.expm_multiply((parameters[k] * spmat_ops[k]), new_state)
    return new_state



def prepare_state_ansatz(cluster_ops_sp, hf_init_sp, parameters):
    """
    It constructs the trial wave function (ansatz) 

    Parameters
    ----------
    cluster_ops_sp: list[Hamiltonian]
        list of spin cluster operators
    
    hf_init_sp: int
        the integer corresponds to the hf_init (The Hartree-Fock state in integer representation) obtained by using
        "qat.fermion.transforms.record_integer".
    
    parameters: List<float>
        the Parameters for the trial wave function to be constructed
    


    Returns
    --------
        curr_state: qat.core.Circuit
            the circuit that represent the trial wave function
    
    """

    prog = Program()
    reg = prog.qalloc(cluster_ops_sp[0].nbqbits)
    for n_term, (term, theta_term) in enumerate(zip(cluster_ops_sp, parameters)):
        init = hf_init_sp if n_term == 0 else 0
        qprog = build_ucc_ansatz([term], init, n_steps=1)
        prog.apply(qprog([theta_term]), reg)
    circ = prog.to_circ()
    curr_state = circ
    return curr_state

In [5]:
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.transforms import transform_to_jw_basis, get_jw_code, recode_integer

#----------------------------------------------Sparse-Hamiltonian-------------------------------------------------------------

geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.99))]
basis = "sto-3g"
spin = 0
charge = 0

(
    rdm1,
    orbital_energies,
    nuclear_repulsion,
    n_electrons,
    one_body_integrals,
    two_body_integrals,
    info,
) = perform_pyscf_computation(geometry=geometry, basis=basis, spin=spin, charge=charge, run_fci=True)

orbital_number = len(orbital_energies)
nqbits = rdm1.shape[0] * 2

# Wrap the hamiltonian data into the `MolecularHamiltonian` class.
mol_h = MolecularHamiltonian(one_body_integrals, two_body_integrals, nuclear_repulsion)

molecule = MoleculeInfo(hamiltonian=mol_h, n_electrons=n_electrons, noons = False,  orbital_energies=orbital_energies)
print("The number of qubits are", molecule.nqbits)
print("The number of electrons are", n_electrons)
print("The number of orbitals are", orbital_number)

hpq, hpqrs = convert_to_h_integrals(one_body_integrals, two_body_integrals)

hamiltonian = ElectronicStructureHamiltonian(
    hpq, hpqrs, constant_coeff=nuclear_repulsion
)

hamiltonian_sparse = hamiltonian.get_matrix(sparse=True)

#print(hamiltonian_sparse)

transformation, code = transform_to_jw_basis, get_jw_code

hamiltonian_sp = transformation(hamiltonian)

hamiltonian_sp_sparse = hamiltonian_sp.get_matrix(sparse=True)

#print(hamiltonian_sp_sparse)



/home/huybinh/.local/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


The number of qubits are 4
The number of electrons are 2
The number of orbitals are 2


In [6]:
#-----------------------------------------------------------Hamiltonian / HF-state---------------------------------------------------------------------

n_elec = n_electrons
transform = "JW"

#-----------------------------------------------------------UCCGSD|Man_Input---------------------------------------------------------------------


single_terms_example = [
    [(1, "Cc", [1, 0]), (-1, "Cc", [0, 1])],
    [(1, "Cc", [2, 0]), (-1, "Cc", [0, 2])],
    [(1, "Cc", [3, 0]), (-1, "Cc", [0, 3])],
    [(1, "Cc", [2, 1]), (-1, "Cc", [1, 2])],
    [(1, "Cc", [3, 1]), (-1, "Cc", [1, 3])],
    [(1, "Cc", [3, 2]), (-1, "Cc", [2, 3])]
    # Add more single term configurations as needed
]

double_terms_example = [
    [(1, "CCcc", [0, 1, 2, 3]), (-1, "CCcc", [3, 2, 1, 0])],
    [(1, "CCcc", [0, 2, 1, 3]), (-1, "CCcc", [3, 1, 2, 0])],
    [(1, "CCcc", [0, 3, 1, 2]), (-1, "CCcc", [2, 1, 3, 0])] 

    # Add more double term configurations as needed
]

#pool_size, cluster_ops, cluster_ops_sp = uccgsd_man(n_elec, orbital_number, transform, single_terms_example, double_terms_example)

pool_size, cluster_ops, cluster_ops_sp = uccgsd(n_elec, orbital_number, transform)

#------------------------------------------Initialize cluster/sparse--------------------------------------


#print(cluster_ops_sparse)

cluster_ops_sparse_shift = []
cluster_shift = []
for i in cluster_ops_sp:
    if i != 0:
        cluster_shift.append(i*1j)
        cluster_ops_sparse_shift.append(i.get_matrix(sparse=True))

cluster_ops_sp = cluster_shift
cluster_ops_sparse = cluster_ops_sparse_shift




In [7]:
    
#----------------------------------------------Reference_ket (Sparse reference state) -------------------------------------------------------------
# HF state sparse

def get_reference_ket(hf_init, nbqbits, transform):


    if transform == "JW":
        hf_init_sp = recode_integer(hf_init, get_jw_code(nbqbits))
    elif transform == "Bravyi-Kitaev":
        hf_init_sp = recode_integer(hf_init, get_bk_code(nbqbits))
    elif transform == "parity_basis":
        hf_init_sp = recode_integer(hf_init, get_parity_code(nbqbits))

    hf_init_sp = recode_integer(hf_init, code(nbqbits))
    ket_hf = binary_repr(hf_init_sp)
    list_ket_hf = [int(c) for c in ket_hf]
    
    state_vector = [1]
    for i in list_ket_hf:
        qubit_vector = [not i, i]
        state_vector = np.kron(state_vector, qubit_vector)

    sparse_reference_state = scipy.sparse.csr_matrix(state_vector, dtype=complex).transpose()

    return sparse_reference_state, hf_init_sp

hf_init = get_hf_ket(molecule.n_electrons, nqbits=molecule.nqbits)

nbqbits = 4

transform = 'JW'


reference_ket, hf_init_sp = get_reference_ket (hf_init, nbqbits, transform)

print("The sparse HF state " ,reference_ket)
#print(hf_init_sp)


#--------------------------------------HF state / energy --------------------------------------------------------


def prepare_hf_state(hf_init_sp, cluster_ops_sp):
    """
    It constructs the Hartree-Fock state (ansatz)

    Parameters
    ----------

    hf_init_sp: int
        the integer corresponds to the hf_init (The Hartree-Fock state in integer representation) obtained by using
        "qat.fermion.transforms.record_integer".

    cluster_ops_sp: list[Hamiltonian]
        list of spin cluster operators
    

    Returns
    --------
        circuit: qat.core.Circuit
            the circuit representing the HF-state
    
    """
    prog = Program()
    nbqbits = cluster_ops_sp[0].nbqbits
    ket_hf = binary_repr(hf_init_sp)
    list_ket_hf = [int(c) for c in ket_hf]
    qb = prog.qalloc(nbqbits)
    for j in range(nbqbits):
        if int(list_ket_hf[j] == 1):
            prog.apply(X, qb[j])
    circuit = prog.to_circ()
    return circuit


def hf_energy(hf_state, hamiltonian_sp):
    """
    Returns the Hartee Fock energy

    Parameters
    ----------

    hf_state: qat.core.Circuit
        the circuit representing the HF state

    hamiltonian_sp: Hamiltonian
        Hamiltonian in the spin representation

    
    Returns
    --------
        res.value: float
            the resulted energy

    """
    qpu = get_default_qpu()
    res = qpu.submit(hf_state.to_job(job_type="OBS", observable=hamiltonian_sp))
    return res.value
    
hf_state = prepare_hf_state(hf_init_sp, cluster_ops_sp)
# This onr returns to circuit 
#hf_state.display()

ref_energy = hf_energy(hf_state, hamiltonian_sp)
# This one returns to the energy
print(ref_energy)
print(" The reference energy of the molecular system is: %12.8f" % ref_energy)


#---------------------------------------------Eigenvals /vecs------------------------------------------------------------------
model = hamiltonian_sp

from scipy.sparse.linalg import eigsh
eigenvalues, eigenvectors = eigsh(model.get_matrix(),k=13)
#print(eigenvalues)
#print(eigenvectors)






The sparse HF state    (12, 0)	(1+0j)
-1.0688975010368331
 The reference energy of the molecular system is:  -1.06889750


In [8]:

#----------------------------------------------UCC_action (display and energy) -------------------------------------------------------------

n_steps =1 
prog = Program()
model = hamiltonian_sp



theta_0 = [np.random.uniform(0, 2*np.pi) for i in range(len(cluster_ops_sp) * n_steps)]
theta = [prog.new_var(float, f"\\theta_{i}") for i in range(len(theta_0))]


#k_lst = [6, 10, hf_init_sp] # List of the excitation
#k_lst = [12, 10, 6]



circ = ucc_action(theta, model, cluster_ops_sp, hf_init_sp)


# Create and store circuits in a list
circuits_store = [circ]


# Display the circuits
for circ_dis in circuits_store:
    circ_dis.display()

# Return to the res.value



### ADAPT VQD

$$2R\left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \hat{H} \left\vert \psi \left( \theta \right)  \right>  +2R\sum^{j-1}_{i=0} \gamma_{i} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{i} \right>  \left< \Phi_{i} \right\vert  \psi \left( \theta \right)  \rangle = \text{Term ground} + \text{Overlap Gradient} $$

#### Step 1: Compute the term ground
- Term ground is the reference ket of HF : $\psi \left( \theta \right)$
- Thus we need to do the sparse of cluster ops and the Hamiltonian 

In [9]:
print(reference_ket) 

  (12, 0)	(1+0j)


In [10]:
print(hamiltonian_sp_sparse)

  (0, 0)	(0.5345224352727271+0j)
  (1, 1)	(-0.05161580933191645+0j)
  (2, 2)	(-0.05161580933191645+0j)
  (3, 3)	(0.016920411200309182+0j)
  (3, 12)	(0.19615797899881998+0j)
  (4, 4)	(-0.5812777667902527+0j)
  (5, 5)	(-0.7403748572446059+0j)
  (6, 6)	(-0.544216878245786+0j)
  (6, 9)	(-0.19615797899881998+0j)
  (7, 7)	(-0.04863950356326996+0j)
  (8, 8)	(-0.5812777667902528+0j)
  (9, 6)	(-0.19615797899881998+0j)
  (9, 9)	(-0.5442168782457861+0j)
  (10, 10)	(-0.740374857244606+0j)
  (11, 11)	(-0.04863950356327007+0j)
  (12, 3)	(0.19615797899881998+0j)
  (12, 12)	(-1.0688975010368331+0j)
  (13, 13)	(-0.6047954583420759+0j)
  (14, 14)	(-0.6047954583420759+0j)
  (15, 15)	(0.5139810494895506+0j)


In [11]:
for i in cluster_ops_sparse:
    print(i)

  (4, 8)	(-1+0j)
  (5, 9)	(-1+0j)
  (6, 10)	(-1+0j)
  (7, 11)	(-1+0j)
  (8, 4)	(1+0j)
  (9, 5)	(1+0j)
  (10, 6)	(1+0j)
  (11, 7)	(1+0j)
  (2, 8)	(-1+0j)
  (3, 9)	(-1+0j)
  (6, 12)	(1+0j)
  (7, 13)	(1+0j)
  (8, 2)	(1+0j)
  (9, 3)	(1+0j)
  (12, 6)	(-1+0j)
  (13, 7)	(-1+0j)
  (1, 8)	(-1+0j)
  (3, 10)	(1+0j)
  (5, 12)	(1+0j)
  (7, 14)	(-1+0j)
  (8, 1)	(1+0j)
  (10, 3)	(-1+0j)
  (12, 5)	(-1+0j)
  (14, 7)	(1+0j)
  (2, 4)	(-1+0j)
  (3, 5)	(-1+0j)
  (4, 2)	(1+0j)
  (5, 3)	(1+0j)
  (10, 12)	(-1+0j)
  (11, 13)	(-1+0j)
  (12, 10)	(1+0j)
  (13, 11)	(1+0j)
  (1, 4)	(-1+0j)
  (3, 6)	(1+0j)
  (4, 1)	(1+0j)
  (6, 3)	(-1+0j)
  (9, 12)	(-1+0j)
  (11, 14)	(1+0j)
  (12, 9)	(1+0j)
  (14, 11)	(-1+0j)
  (1, 2)	(-1+0j)
  (2, 1)	(1+0j)
  (5, 6)	(-1+0j)
  (6, 5)	(1+0j)
  (9, 10)	(-1+0j)
  (10, 9)	(1+0j)
  (13, 14)	(-1+0j)
  (14, 13)	(1+0j)
  (10, 12)	(1+0j)
  (11, 13)	(1+0j)
  (12, 10)	(-1+0j)
  (13, 11)	(-1+0j)
  (9, 12)	(1+0j)
  (11, 14)	(-1+0j)
  (12, 9)	(-1+0j)
  (14, 11)	(1+0j)
  (6, 12)	(1+0j)
  (7, 13)	(

In [12]:


def return_gradient_ground(cluster_ops_sparse, hamiltonian_sparse, curr_state):
    """
    Compute analytically the gradient for all fermionic cluster operators ("cluster_ops_sparse").

    Parameters
    -----------
    cluster_ops_sparse: List<transposable linear operator>
        the sparse fermionic operators

    hamiltonian_sparse: ndarray
        the hamiltonian operator
    
    curr_state: ndarray
        the current state

    Returns
    --------
    list_grad: List<float>
        the gradients for all the fermionic cluster operators
    
    curr_norm: float
        the magnitude of the gradients
    
    next_deriv: float
        the maximum gradient (in absolute value)
    
    next_index: int
        the index of the operator with the maximum gradient

    """
    
    list_grad_g = []
    curr_norm_g = 0
    next_deriv_g = 0
    next_index_g = 0
    sig = hamiltonian_sparse.dot(curr_state)
    
    for oi in range(len(cluster_ops_sparse)):
        op_a = cluster_ops_sparse[oi]
        gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state)))
        assert gi.shape == (1, 1)
        gi = gi[0, 0]
        assert np.isclose(gi.imag, 0)
        gi = gi.real
        
        list_grad_g.append(abs(gi))
        curr_norm_g += gi * gi
        
        if abs(gi) > abs(next_deriv_g):
            next_deriv_g = gi
            next_index_g = oi
            
    return list_grad_g, curr_norm_g, next_deriv_g, next_index_g


curr_state = reference_ket

#list_grad_g, curr_norm_g, next_deriv_g, next_index_g = return_gradient_ground(cluster_ops_sparse, hamiltonian_sparse, curr_state)

#print(" Norm of the gradients in current iteration = %12.8f" % curr_norm_g)
#print(" Max gradient in current iteration= %12.8f" % next_deriv_g)
#print(" Index of the Max gradient in current iteration= ", next_index_g)

#print("The list of gradient values for the ground state are", *list_grad_g, sep='\n')


In [13]:
# I put all of them in a loop for the ground term
n_max_grads = 1
optimizer = 'COBYLA'  
max_external_iterations=30
type_conver = 'norm'
threshold_needed = 1e-3
tolerance = 10**(-7) 

result = {}
print("threshold needed for convergence", threshold_needed)
print("Max_external_iterations:", max_external_iterations)
print("how many maximum gradient are selected", n_max_grads)
print("The optimizer method used:", optimizer)
print("Tolerance for reaching convergence", tolerance)



curr_state = hf_state
curr_state_open_f = 1.0 * reference_ket
prev_norm = 0.0



for n_iter in range(0, max_external_iterations):
    print("\n\n\n")
    print(
        " --------------------------------------------------------------------------"
    )
    print("                     Fermionic_ADAPT-VQE iteration: ", n_iter)
    print(
        " --------------------------------------------------------------------------"
    )
    # few commands to initialize at each new iteration
    list_grad_g, curr_norm_g, next_deriv_g, next_index_g = return_gradient_ground(
        cluster_ops_sparse, hamiltonian_sparse, curr_state_open_f
        )


    # the norm from the current iteration:
    curr_norm_g = np.sqrt(curr_norm_g)
    print(" Norm of the gradients in current iteration = %12.8f" % curr_norm_g)
    print(" Max gradient in current iteration= %12.8f" % next_deriv_g)
    print(" Index of the Max gradient in current iteration= ", next_index_g)
    
    converged = False
    if type_conver == "norm":
        if curr_norm_g < threshold_needed:
            converged = True
    else:
        print(" type convergence is not defined")
        exit()
    if converged or (abs(curr_norm_g - prev_norm) < 10 ** (-8)):
        print("Convergence is done")
        result["Number_operators"] = len(ansatz_ops)
        result["final_norm"] = curr_norm_g
        result["parameters"] = parameters_ansatz
        print(" -----------Final ansatz----------- ")
        print(" *final converged energy iteration is %20.12f" % opt_result.fun)
        result["final_energy_last_iteration"] = opt_result.fun
        break

    ansatz_ops = cluster_ops_sp
    ansatz_mat =  cluster_ops_sparse

    opt_result = scipy.optimize.minimize(
        lambda parameters: ucc_action_energy(
            hamiltonian_sp, ansatz_ops, hf_init_sp, parameters
        ),
        x0=theta_0,
        method=optimizer,
        tol=tolerance,
        options={"maxiter": 100000, "disp": True},
    )
    xlist_0 = opt_result.x
    print(" Finished energy iteration_i: %20.12f" % opt_result.fun)
    print(" -----------New ansatz created----------- ")
    print(" %4s \t%s \t%s" % ("#", "Coefficients", "Term"))
    parameters_ansatz = []

    for si in range(len(cluster_ops_sp)):
        print(" %4i \t%f " % (si, xlist_0[si]))
        parameters_ansatz.append(xlist_0[si])

    curr_state = prepare_state_ansatz(ansatz_ops, hf_init_sp, parameters_ansatz) 
    curr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz) # this is for the current state of the HF
    prev_norm = curr_norm_g








threshold needed for convergence 0.001
Max_external_iterations: 30
how many maximum gradient are selected 1
The optimizer method used: COBYLA
Tolerance for reaching convergence 1e-07




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQE iteration:  0
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration =   0.39231596
 Max gradient in current iteration=   0.39231596
 Index of the Max gradient in current iteration=  10
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	-0.077236 
    1 	1.497634 
    2 	6.155115 
    3 	5.152275 
    4 	1.416193 
    5 	3.237988 
    6 	6.033823 
    7 	2.420255 
    8 	4.776762 
    9 	2.307069 
   10 	4.892870 
   11 	5.774474 
   12 	5.523159 
   13 	0.296102 
   14 	6.206516 
   15 	6.179529 
   16 	3.011840 
   17 	3.295005 
   18 	2.186804 
 

#### Step 2: Overlap gradient for the ground state 

$$2R\left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \hat{H} \left\vert \psi \left( \theta \right)  \right> + 2R \times \gamma \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{0} \right>  \left< \Phi_{0} \right\vert  \psi \left( \theta \right)  \rangle$$

- $\Phi_{0}$ is the state vector (sparse) of the ground state

In [14]:
qpu = get_default_qpu()
method = "BFGS"

model = hamiltonian_sp
nqbits = model.nbqbits

energy_lists = {f"energy_circ_{i}": {method: []} for i in range(len(circuits_store))}

def get_statevector_matrix(result, nqbits):

    statevector_mat = np.zeros((2**nqbits), np.complex128)
    for sample in result:
        statevector_mat[sample.state.int] = sample.amplitude
    return statevector_mat


def opt_funct(circuits, model, qpu, nqbits, energy_lists):
    
    def input_funct(x):
        total_energy = 0
        for i, circ in enumerate(circuits):
           
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)
            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)

            total_energy = energy

        return total_energy
    
    def callback(x):
        
        for i, circ in enumerate(circuits):
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)

            

            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)
    
    
    return input_funct, callback


circuits = circuits_store

input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists)
options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
#options = {"disp": True, "maxiter": 1000, "gtol": 1e-5}
Optimizer_0 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)

opt_circ = circuits_store[0].bind_variables({k: v for k, v in zip(sorted(circuits_store[0].get_variables()), Optimizer_0.x)})
result = qpu.submit(opt_circ.to_job())
stv_ground_mat= get_statevector_matrix(result, nqbits)



#--------------------------------------------Sparse statevector matrix-------------------------------------------------------------


stv_ground_mat_sparse = scipy.sparse.csr_matrix(stv_ground_mat, dtype=complex).transpose()
print(stv_ground_mat_sparse)

"""
#stv_ground_mat_sparse_conj = scipy.sparse.csr_matrix(stv_ground_mat, dtype=complex).transpose().conj()
#print(stv_ground_mat_sparse_conj)

#Tested checking


curr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz)

print("Dimensions of curr_state_open_f:", curr_state_open_f.shape)

#print(curr_state_open_f)

curr_state_open_g = prepare_adapt_state(stv_ground_mat_sparse, ansatz_mat, parameters_ansatz) # this alwyas need to input the transpose

print("Dimensions of curr_state_open_g:", curr_state_open_g.shape)

#print(curr_state_open_g)



sig = hamiltonian_sparse.dot(curr_state_open_f)
#print(sig)

ovl = hamiltonian_sparse.dot(curr_state_open_g)
print(ovl)


for oi in range(len(cluster_ops_sparse)):
    op_a = cluster_ops_sparse[oi]

    gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state_open_f))) + 22* (ovl.transpose().conj().dot(op_a.dot(curr_state_open_g)))
    print(gi)

""" 


         Current function value: -1.103248
         Iterations: 17
         Function evaluations: 644
         Gradient evaluations: 29
  (3, 0)	(-0.17249001436748537+0j)
  (5, 0)	(-1.3679977647722272e-07+0j)
  (6, 0)	(1.2299112547853308e-08+0j)
  (9, 0)	(3.878400350876273e-09+0j)
  (10, 0)	(3.523274780259377e-08+0j)
  (12, 0)	(0.9850112664043212+0j)


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


'\n#stv_ground_mat_sparse_conj = scipy.sparse.csr_matrix(stv_ground_mat, dtype=complex).transpose().conj()\n#print(stv_ground_mat_sparse_conj)\n\n#Tested checking\n\n\ncurr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz)\n\nprint("Dimensions of curr_state_open_f:", curr_state_open_f.shape)\n\n#print(curr_state_open_f)\n\ncurr_state_open_g = prepare_adapt_state(stv_ground_mat_sparse, ansatz_mat, parameters_ansatz) # this alwyas need to input the transpose\n\nprint("Dimensions of curr_state_open_g:", curr_state_open_g.shape)\n\n#print(curr_state_open_g)\n\n\n\nsig = hamiltonian_sparse.dot(curr_state_open_f)\n#print(sig)\n\novl = hamiltonian_sparse.dot(curr_state_open_g)\nprint(ovl)\n\n\nfor oi in range(len(cluster_ops_sparse)):\n    op_a = cluster_ops_sparse[oi]\n\n    gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state_open_f))) + 22* (ovl.transpose().conj().dot(op_a.dot(curr_state_open_g)))\n    print(gi)\n\n'

In [15]:


def return_gradient_overlap_0(cluster_ops_sparse, hamiltonian_sparse, curr_state, phi_0, gamma_0):
    """
    Compute analytically the gradient for all fermionic cluster operators ("cluster_ops_sparse").

    Parameters
    -----------
    cluster_ops_sparse: List<transposable linear operator>
        the sparse fermionic operators

    hamiltonian_sparse: ndarray
        the hamiltonian operator
    
    curr_state: ndarray
        the current state

    Returns
    --------
    list_grad: List<float>
        the gradients for all the fermionic cluster operators
    
    curr_norm: float
        the magnitude of the gradients
    
    next_deriv: float
        the maximum gradient (in absolute value)
    
    next_index: int
        the index of the operator with the maximum gradient

    """
    
    list_grad_g = []
    curr_norm_g = 0
    next_deriv_g = 0
    next_index_g = 0
    sig = hamiltonian_sparse.dot(curr_state)
    ovl_0 = hamiltonian_sparse.dot(phi_0)
    
    for oi in range(len(cluster_ops_sparse)):
        op_a = cluster_ops_sparse[oi]
        gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state))) + 2 * gamma_0*(ovl_0.transpose().conj().dot(op_a.dot(phi_0)))
        assert gi.shape == (1, 1)
        gi = gi[0, 0]
        assert np.isclose(gi.imag, 0)
        gi = gi.real
        
        list_grad_g.append(abs(gi))
        curr_norm_g += gi * gi
        
        if abs(gi) > abs(next_deriv_g):
            next_deriv_g = gi
            next_index_g = oi
            
    return list_grad_g, curr_norm_g, next_deriv_g, next_index_g

In [16]:
n_max_grads = 1
optimizer = 'COBYLA'  
max_external_iterations=30
type_conver = 'norm'
threshold_needed = 1e-3
tolerance = 10**(-7) 

result = {}
print("Threshold needed for convergence", threshold_needed)
print("Max_external_iterations:", max_external_iterations)
print("how many maximum gradient are selected", n_max_grads)
print("The optimizer method used:", optimizer)
print("Tolerance for reaching convergence", tolerance)



curr_state = hf_state
curr_state_open_f = 1.0 * reference_ket
gamma_0 = 11
curr_state_open_g = gamma_0 * reference_ket
prev_norm = 0.0



for n_iter in range(0, max_external_iterations):
    print("\n\n\n")
    print(
        " --------------------------------------------------------------------------"
    )
    print("                     Fermionic_ADAPT-VQD iteration: ", n_iter)
    print(
        " --------------------------------------------------------------------------"
    )
    # few commands to initialize at each new iteration
    list_grad_ovl_1, curr_norm_ovl_1, next_deriv_ovl_1, next_index_ovl_1 = return_gradient_overlap_0(
        cluster_ops_sparse, hamiltonian_sparse, curr_state_open_f, curr_state_open_g, gamma_0
        )


    # the norm from the current iteration:
    curr_norm_ovl_1 = np.sqrt(curr_norm_ovl_1)
    print(" Norm of the gradients in current iteration = %12.8f" % curr_norm_ovl_1)
    print(" Max gradient in current iteration= %12.8f" % next_deriv_ovl_1)
    print(" Index of the Max gradient in current iteration= ", next_index_ovl_1)
    
    converged = False
    if type_conver == "norm":
        if curr_norm_ovl_1 < threshold_needed:
            converged = True
    else:
        print(" type convergence is not defined")
        exit()
    if converged or (abs(curr_norm_ovl_1 - prev_norm) < 10 ** (-8)):
        print("Convergence is done")
        result["Number_operators"] = len(ansatz_ops)
        result["final_norm"] = curr_norm_ovl_1
        result["parameters"] = parameters_ansatz
        print(" -----------Final ansatz----------- ")
        print(" *final converged energy iteration is %20.12f" % Optimizer_0.fun)
        result["final_energy_last_iteration"] = Optimizer_0.fun
        break

    ansatz_ops = cluster_ops_sp
    ansatz_mat =  cluster_ops_sparse


    circuits = circuits_store
    
    input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists)
    options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
    #options = {"disp": True, "maxiter": 1000, "gtol": 1e-5}
    Optimizer_0 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)
    xlist_ovl_0 = Optimizer_0.x
    
    
    print(" Finished energy iteration_i: %20.12f" % Optimizer_0.fun)
    print(" -----------New ansatz created----------- ")
    print(" %4s \t%s \t%s" % ("#", "Coefficients", "Term"))
    parameters_ansatz = []

    for si in range(len(cluster_ops_sp)):
        print(" %4i \t%f " % (si, xlist_ovl_0[si]))
        parameters_ansatz.append(xlist_ovl_0[si])

    curr_state = prepare_state_ansatz(ansatz_ops, hf_init_sp, parameters_ansatz) 
    curr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz) # this is for the current state of the HF
    curr_state_open_g = prepare_adapt_state(stv_ground_mat_sparse, ansatz_mat, parameters_ansatz)
    prev_norm = curr_norm_ovl_1

Threshold needed for convergence 0.001
Max_external_iterations: 30
how many maximum gradient are selected 1
The optimizer method used: COBYLA
Tolerance for reaching convergence 1e-07




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  0
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 522.56485605
 Max gradient in current iteration= 522.56485605
 Index of the Max gradient in current iteration=  10


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -1.103248
         Iterations: 20
         Function evaluations: 968
         Gradient evaluations: 44
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.609974 
    2 	5.658527 
    3 	4.987023 
    4 	1.546689 
    5 	2.949693 
    6 	5.137053 
    7 	0.985080 
    8 	3.397946 
    9 	2.028795 
   10 	5.033995 
   11 	4.445425 
   12 	4.565163 
   13 	0.289100 
   14 	6.842386 
   15 	6.558375 
   16 	2.233131 
   17 	2.877767 
   18 	2.221659 
   19 	1.632232 
   20 	0.044322 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  1
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration =   5.88157465
 Max gradient in current iteration=   3.17535826
 Index of the Max gradient in current iteration=  8


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -1.103248
         Iterations: 17
         Function evaluations: 880
         Gradient evaluations: 40
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032303 
    1 	0.609974 
    2 	5.658527 
    3 	4.987024 
    4 	1.546689 
    5 	2.949692 
    6 	5.137055 
    7 	0.985080 
    8 	3.397947 
    9 	2.028794 
   10 	5.033995 
   11 	4.445425 
   12 	4.565163 
   13 	0.289100 
   14 	6.842387 
   15 	6.558374 
   16 	2.233131 
   17 	2.877767 
   18 	2.221659 
   19 	1.632232 
   20 	0.044322 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  2
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration =   5.88157880
 Max gradient in current iteration=   3.17536481
 Index of the Max gradient in current iteration=  8


...

         Current function value: -1.103248
         Iterations: 16
         Function evaluations: 484
         Gradient evaluations: 22
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.609975 
    2 	5.658527 
    3 	4.987024 
    4 	1.546689 
    5 	2.949693 
    6 	5.137055 
    7 	0.985080 
    8 	3.397946 
    9 	2.028795 
   10 	5.033995 
   11 	4.445424 
   12 	4.565163 
   13 	0.289100 
   14 	6.842387 
   15 	6.558374 
   16 	2.233131 
   17 	2.877766 
   18 	2.221658 
   19 	1.632231 
   20 	0.044323 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  28
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration =   5.88158513
 Max gradient in current iteration=   3.17536965
 Index of the Max gradient in current iteration=  8


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -1.103248
         Iterations: 17
         Function evaluations: 484
         Gradient evaluations: 22
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.609974 
    2 	5.658527 
    3 	4.987024 
    4 	1.546690 
    5 	2.949692 
    6 	5.137054 
    7 	0.985080 
    8 	3.397946 
    9 	2.028795 
   10 	5.033995 
   11 	4.445425 
   12 	4.565163 
   13 	0.289100 
   14 	6.842387 
   15 	6.558374 
   16 	2.233131 
   17 	2.877766 
   18 	2.221659 
   19 	1.632232 
   20 	0.044321 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  29
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration =   5.88158696
 Max gradient in current iteration=   3.17537068
 Index of the Max gradient in current iteration=  8


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -1.103248
         Iterations: 17
         Function evaluations: 572
         Gradient evaluations: 26
 Finished energy iteration_i:      -1.103247661224
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032301 
    1 	0.609975 
    2 	5.658527 
    3 	4.987024 
    4 	1.546689 
    5 	2.949693 
    6 	5.137054 
    7 	0.985080 
    8 	3.397946 
    9 	2.028794 
   10 	5.033995 
   11 	4.445425 
   12 	4.565163 
   13 	0.289100 
   14 	6.842386 
   15 	6.558374 
   16 	2.233131 
   17 	2.877767 
   18 	2.221660 
   19 	1.632232 
   20 	0.044323 


#### Step 3: Overlap gradient for the first excited state 

$$2R\left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \hat{H} \left\vert \psi \left( \theta \right)  \right> + 2R \times \gamma_{0} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{0} \right>  \left< \Phi_{0} \right\vert  \psi \left( \theta \right)  \rangle +  2R \times \gamma_{1} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{1} \right>  \left< \Phi_{1} \right\vert  \psi \left( \theta \right)  \rangle $$

In [17]:
opt_circ = circuits_store[0].bind_variables({k: v for k, v in zip(sorted(circuits_store[0].get_variables()), Optimizer_0.x)})
result = qpu.submit(opt_circ.to_job())

beta_1 = 33

def get_statevector_matrix(result, nqbits):

    statevector_mat = np.zeros((2**nqbits), np.complex128)
    for sample in result:
        statevector_mat[sample.state.int] = sample.amplitude
    return statevector_mat

def overlap(circ, def_stv):
    qpu = get_default_qpu()
    res = qpu.submit(circ.to_job())
    statevector = get_statevector_matrix(res, nqbits)
    
    # Check if the statevector is not None
    if statevector is not None:
        overlap_vals = abs(np.vdot(def_stv, statevector)) ** 2
        return overlap_vals
    else:
        return None

stv_ground_mat= get_statevector_matrix(result, nqbits)


energy_lists = {f"energy_circ_{i}": {method: []} for i in range(len(circuits_store))}

def opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1):
    
    def input_funct(x):
        total_energy = 0
        for i, circ in enumerate(circuits):
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)

            

            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)
            overlap_inp = overlap(bound_circ, stv_ground_mat)
                

            total_energy = energy + beta_1 * overlap_inp 

        return total_energy
    

    def callback(x):
        
        for i, circ in enumerate(circuits):
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)

            

            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)
            overlap_inp = overlap(bound_circ, stv_ground_mat)


    return input_funct, callback

circuits = circuits_store

input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1)
options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
Optimizer_1 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)

opt_circ = circuits_store[0].bind_variables({k: v for k, v in zip(sorted(circuits_store[0].get_variables()), Optimizer_1.x)})
result = qpu.submit(opt_circ.to_job())
stv_exci_1_mat = get_statevector_matrix(result, nqbits)


#--------------------------------------------Sparse statevector matrix-------------------------------------------------------------


stv_exci_1_mat_sparse = scipy.sparse.csr_matrix(stv_exci_1_mat, dtype=complex).transpose()
print(stv_exci_1_mat_sparse)

         Current function value: -0.740375
         Iterations: 14
         Function evaluations: 971
         Gradient evaluations: 44
  (3, 0)	(-6.535541703764532e-08+0j)
  (5, 0)	(-0.02702528435781302+0j)
  (6, 0)	(-0.28903200696587666+0j)
  (9, 0)	(-0.2890320378012201+0j)
  (10, 0)	(-0.9122448213495468+0j)
  (12, 0)	(1.3875303175522877e-07+0j)


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [18]:


def return_gradient_overlap_1(cluster_ops_sparse, hamiltonian_sparse, curr_state, phi_0,  phi_1, gamma_0, gamma_1):
    """
    Compute analytically the gradient for all fermionic cluster operators ("cluster_ops_sparse").

    Parameters
    -----------
    cluster_ops_sparse: List<transposable linear operator>
        the sparse fermionic operators

    hamiltonian_sparse: ndarray
        the hamiltonian operator
    
    curr_state: ndarray
        the current state

    Returns
    --------
    list_grad: List<float>
        the gradients for all the fermionic cluster operators
    
    curr_norm: float
        the magnitude of the gradients
    
    next_deriv: float
        the maximum gradient (in absolute value)
    
    next_index: int
        the index of the operator with the maximum gradient

    """
    
    list_grad_g = []
    curr_norm_g = 0
    next_deriv_g = 0
    next_index_g = 0
    sig = hamiltonian_sparse.dot(curr_state)
    ovl_0 = hamiltonian_sparse.dot(phi_0)
    ovl_1 = hamiltonian_sparse.dot(phi_1)
    
    for oi in range(len(cluster_ops_sparse)):
        op_a = cluster_ops_sparse[oi]
        gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state))) + 2 * gamma_0*(ovl_0.transpose().conj().dot(op_a.dot(phi_0))) + 2 * gamma_1*(ovl_1.transpose().conj().dot(op_a.dot(phi_1)))
        assert gi.shape == (1, 1)
        gi = gi[0, 0]
        assert np.isclose(gi.imag, 0)
        gi = gi.real
        
        list_grad_g.append(abs(gi))
        curr_norm_g += gi * gi
        
        if abs(gi) > abs(next_deriv_g):
            next_deriv_g = gi
            next_index_g = oi
            
    return list_grad_g, curr_norm_g, next_deriv_g, next_index_g

In [19]:
n_max_grads = 1
optimizer = 'COBYLA'  
max_external_iterations=30
type_conver = 'norm'
threshold_needed = 1e-3
tolerance = 10**(-7) 

result = {}
print("Threshold needed for convergence", threshold_needed)
print("Max_external_iterations:", max_external_iterations)
print("how many maximum gradient are selected", n_max_grads)
print("The optimizer method used:", optimizer)
print("Tolerance for reaching convergence", tolerance)



curr_state = hf_state
curr_state_open_f = 1.0 * reference_ket
gamma_0 = 11
curr_state_open_g = gamma_0 * reference_ket
gamma_1 = 22
curr_state_open_h = gamma_1 * reference_ket

prev_norm = 0.0



for n_iter in range(0, max_external_iterations):
    print("\n\n\n")
    print(
        " --------------------------------------------------------------------------"
    )
    print("                     Fermionic_ADAPT-VQD iteration: ", n_iter)
    print(
        " --------------------------------------------------------------------------"
    )
    # few commands to initialize at each new iteration
    list_grad_ovl_2, curr_norm_ovl_2, next_deriv_ovl_2, next_index_ovl_2 = return_gradient_overlap_1(
        cluster_ops_sparse, hamiltonian_sparse, curr_state_open_f, curr_state_open_g, curr_state_open_h, gamma_0, gamma_1
        )


    # the norm from the current iteration:
    curr_norm_ovl_2 = np.sqrt(curr_norm_ovl_2)
    print(" Norm of the gradients in current iteration = %12.8f" % curr_norm_ovl_2)
    print(" Max gradient in current iteration= %12.8f" % next_deriv_ovl_2)
    print(" Index of the Max gradient in current iteration= ", next_index_ovl_2)
    
    converged = False
    if type_conver == "norm":
        if curr_norm_ovl_2 < threshold_needed:
            converged = True
    else:
        print(" type convergence is not defined")
        exit()
    if converged or (abs(curr_norm_ovl_2 - prev_norm) < 10 ** (-8)):
        print("Convergence is done")
        result["Number_operators"] = len(ansatz_ops)
        result["final_norm"] = curr_norm_ovl_2
        result["parameters"] = parameters_ansatz
        print(" -----------Final ansatz----------- ")
        print(" *final converged energy iteration is %20.12f" % Optimizer_1.fun)
        result["final_energy_last_iteration"] = Optimizer_1.fun
        break

    ansatz_ops = cluster_ops_sp
    ansatz_mat =  cluster_ops_sparse


    circuits = circuits_store
    
    input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1)
    options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
    Optimizer_1 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)
    xlist_ovl_1 = Optimizer_1.x
    
    
    print(" Finished energy iteration_i: %20.12f" % Optimizer_1.fun)
    print(" -----------New ansatz created----------- ")
    print(" %4s \t%s \t%s" % ("#", "Coefficients", "Term"))
    parameters_ansatz = []

    for si in range(len(cluster_ops_sp)):
        print(" %4i \t%f " % (si, xlist_ovl_1[si]))
        parameters_ansatz.append(xlist_ovl_1[si])

    curr_state = prepare_state_ansatz(ansatz_ops, hf_init_sp, parameters_ansatz) 
    curr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz) # this is for the current state of the HF
    curr_state_open_g = prepare_adapt_state(stv_ground_mat_sparse, ansatz_mat, parameters_ansatz)
    curr_state_open_h = prepare_adapt_state(stv_exci_1_mat_sparse, ansatz_mat, parameters_ansatz)

    prev_norm = curr_norm_g

Threshold needed for convergence 0.001
Max_external_iterations: 30
how many maximum gradient are selected 1
The optimizer method used: COBYLA
Tolerance for reaching convergence 1e-07




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  0
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 4699.94517681
 Max gradient in current iteration= 4699.94517681
 Index of the Max gradient in current iteration=  10


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -0.740375
         Iterations: 9
         Function evaluations: 1531
         Gradient evaluations: 69
 Finished energy iteration_i:      -0.740374857245
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.638619 
    2 	5.285309 
    3 	5.061731 
    4 	1.456292 
    5 	2.949427 
    6 	5.190942 
    7 	0.912765 
    8 	3.471821 
    9 	2.085492 
   10 	4.968301 
   11 	4.302893 
   12 	4.433787 
   13 	0.208792 
   14 	6.077331 
   15 	5.951486 
   16 	2.345011 
   17 	2.827678 
   18 	2.303245 
   19 	0.885762 
   20 	0.188455 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  1
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 4181.28543612
 Max gradient in current iteration= 4181.27217317
 Index of the Max gradient in current iteration=  10
Optimization 

/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -0.740375
         Iterations: 12
         Function evaluations: 913
         Gradient evaluations: 41
 Finished energy iteration_i:      -0.740374857245
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.638619 
    2 	5.285309 
    3 	5.061731 
    4 	1.456291 
    5 	2.949427 
    6 	5.190942 
    7 	0.912765 
    8 	3.471820 
    9 	2.085492 
   10 	4.968301 
   11 	4.302893 
   12 	4.433787 
   13 	0.208792 
   14 	6.077331 
   15 	5.951486 
   16 	2.345011 
   17 	2.827678 
   18 	2.303244 
   19 	0.885762 
   20 	0.188455 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  3
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 4181.28543591
 Max gradient in current iteration= 4181.27217296
 Index of the Max gradient in current iteration=  10


...

         Current function value: -0.740375
         Iterations: 10
         Function evaluations: 1180
         Gradient evaluations: 53
 Finished energy iteration_i:      -0.740374857245
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.638619 
    2 	5.285309 
    3 	5.061731 
    4 	1.456292 
    5 	2.949427 
    6 	5.190942 
    7 	0.912765 
    8 	3.471821 
    9 	2.085492 
   10 	4.968301 
   11 	4.302893 
   12 	4.433787 
   13 	0.208792 
   14 	6.077331 
   15 	5.951486 
   16 	2.345011 
   17 	2.827678 
   18 	2.303244 
   19 	0.885762 
   20 	0.188455 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  28
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 4181.28543635
 Max gradient in current iteration= 4181.27217339
 Index of the Max gradient in current iteration=  10


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -0.740375
         Iterations: 10
         Function evaluations: 374
         Gradient evaluations: 17
 Finished energy iteration_i:      -0.740374857245
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.638619 
    2 	5.285309 
    3 	5.061731 
    4 	1.456291 
    5 	2.949427 
    6 	5.190942 
    7 	0.912765 
    8 	3.471820 
    9 	2.085492 
   10 	4.968301 
   11 	4.302893 
   12 	4.433787 
   13 	0.208792 
   14 	6.077331 
   15 	5.951486 
   16 	2.345011 
   17 	2.827678 
   18 	2.303244 
   19 	0.885762 
   20 	0.188455 




 --------------------------------------------------------------------------
                     Fermionic_ADAPT-VQD iteration:  29
 --------------------------------------------------------------------------
 Norm of the gradients in current iteration = 4181.28543725
 Max gradient in current iteration= 4181.27217429
 Index of the Max gradient in current iteration=  10


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: -0.740375
         Iterations: 11
         Function evaluations: 672
         Gradient evaluations: 30
 Finished energy iteration_i:      -0.740374857245
 -----------New ansatz created----------- 
    # 	Coefficients 	Term
    0 	0.032302 
    1 	0.638619 
    2 	5.285309 
    3 	5.061731 
    4 	1.456292 
    5 	2.949427 
    6 	5.190942 
    7 	0.912765 
    8 	3.471821 
    9 	2.085492 
   10 	4.968301 
   11 	4.302893 
   12 	4.433787 
   13 	0.208792 
   14 	6.077331 
   15 	5.951486 
   16 	2.345011 
   17 	2.827678 
   18 	2.303245 
   19 	0.885762 
   20 	0.188455 


#### Step 3: Overlap gradient for the second excited state 

$$2R\left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \hat{H} \left\vert \psi \left( \theta \right)  \right> + 2R \times \gamma_{0} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{0} \right>  \left< \Phi_{0} \right\vert  \psi \left( \theta \right)  \rangle +  2R \times \gamma_{1} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{1} \right>  \left< \Phi_{1} \right\vert  \psi \left( \theta \right)  \rangle + 2R \times \gamma_{2} \left< \psi \left( \theta \right)  \right\vert  \hat{A}_{m} \left\vert \Phi_{2} \right>  \left< \Phi_{2} \right\vert  \psi \left( \theta \right)  \rangle $$

In [20]:
opt_circ = circuits_store[0].bind_variables({k: v for k, v in zip(sorted(circuits_store[0].get_variables()), Optimizer_1.x)})
result = qpu.submit(opt_circ.to_job())

beta_2 = 44

def get_statevector_matrix(result, nqbits):

    statevector_mat = np.zeros((2**nqbits), np.complex128)
    for sample in result:
        statevector_mat[sample.state.int] = sample.amplitude
    return statevector_mat

def overlap(circ, def_stv):
    qpu = get_default_qpu()
    res = qpu.submit(circ.to_job())
    statevector = get_statevector_matrix(res, nqbits)
    
    # Check if the statevector is not None
    if statevector is not None:
        overlap_vals = abs(np.vdot(def_stv, statevector)) ** 2
        return overlap_vals
    else:
        return None

stv_exci_mat= get_statevector_matrix(result, nqbits)


energy_lists = {f"energy_circ_{i}": {method: []} for i in range(len(circuits_store))}

def opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1, beta_2):
    
    def input_funct(x):
        total_energy = 0
        for i, circ in enumerate(circuits):
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)


            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)
            overlap_inp = overlap(bound_circ, stv_ground_mat)
            overlap_inp_exci = overlap(bound_circ, stv_exci_mat)
                


            total_energy = energy + beta_1 * overlap_inp + beta_2 * overlap_inp_exci

        return total_energy
    

    def callback(x):
        
        for i, circ in enumerate(circuits):
            bound_circ = circ.bind_variables({k: v for k, v in zip(sorted(circ.get_variables()), x)})
            job_exci_inp = bound_circ.to_job(job_type="OBS", observable=model, nbshots=0)
            result = qpu.submit(job_exci_inp)

            

            energy = result.value
            energy_lists[f"energy_circ_{i}"][method].append(energy)
            overlap_inp = overlap(bound_circ, stv_ground_mat)
            overlap_inp_exci = overlap(bound_circ, stv_exci_mat)


    return input_funct, callback
    


circuits = circuits_store



input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1, beta_2)
options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
Optimizer_2 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)
opt_circ = circuits_store[0].bind_variables({k: v for k, v in zip(sorted(circuits_store[0].get_variables()), Optimizer_2.x)})

result = qpu.submit(opt_circ.to_job())
stv_exci_2_mat = get_statevector_matrix(result, nqbits)


#--------------------------------------------Sparse statevector matrix-------------------------------------------------------------


stv_exci_2_mat_sparse = scipy.sparse.csr_matrix(stv_exci_2_mat, dtype=complex).transpose()
print(stv_exci_2_mat_sparse)

         Current function value: -0.740375
         Iterations: 17
         Function evaluations: 1110
         Gradient evaluations: 50
  (3, 0)	(1.985641882862321e-08+0j)
  (5, 0)	(0.9555834968614103+0j)
  (6, 0)	(-0.19691775873626213+0j)
  (9, 0)	(-0.19691802363259783+0j)
  (10, 0)	(0.09647211404839012+0j)
  (12, 0)	(-1.7272724333727144e-07+0j)


/home/huybinh/.local/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [21]:


def return_gradient_overlap_2(cluster_ops_sparse, hamiltonian_sparse, curr_state, phi_0, phi_1, phi_2, gamma_0, gamma_1, gamma_2):
    """
    Compute analytically the gradient for all fermionic cluster operators ("cluster_ops_sparse").

    Parameters
    -----------
    cluster_ops_sparse: List<transposable linear operator>
        the sparse fermionic operators

    hamiltonian_sparse: ndarray
        the hamiltonian operator
    
    curr_state: ndarray
        the current state

    Returns
    --------
    list_grad: List<float>
        the gradients for all the fermionic cluster operators
    
    curr_norm: float
        the magnitude of the gradients
    
    next_deriv: float
        the maximum gradient (in absolute value)
    
    next_index: int
        the index of the operator with the maximum gradient

    """
    
    list_grad_g = []
    curr_norm_g = 0
    next_deriv_g = 0
    next_index_g = 0
    sig = hamiltonian_sparse.dot(curr_state)
    ovl_0 = hamiltonian_sparse.dot(phi_0)
    ovl_1 = hamiltonian_sparse.dot(phi_1)
    ovl_2 = hamiltonian_sparse.dot(phi_2)
    
    for oi in range(len(cluster_ops_sparse)):
        op_a = cluster_ops_sparse[oi]
        gi = 2 * (sig.transpose().conj().dot(op_a.dot(curr_state))) + 2 * gamma_0*(ovl_0.transpose().conj().dot(op_a.dot(phi_0))) + 2 * gamma_1*(ovl_1.transpose().conj().dot(op_a.dot(phi_1))) + 2 * gamma_2*(ovl_2.transpose().conj().dot(op_a.dot(phi_2)))
        assert gi.shape == (1, 1)
        gi = gi[0, 0]
        assert np.isclose(gi.imag, 0)
        gi = gi.real
        
        list_grad_g.append(abs(gi))
        curr_norm_g += gi * gi
        
        if abs(gi) > abs(next_deriv_g):
            next_deriv_g = gi
            next_index_g = oi
            
    return list_grad_g, curr_norm_g, next_deriv_g, next_index_g

In [ ]:
n_max_grads = 1
optimizer = 'COBYLA'  
max_external_iterations=30
type_conver = 'norm'
threshold_needed = 1e-3
tolerance = 10**(-7) 

result = {}
print("Threshold needed for convergence", threshold_needed)
print("Max_external_iterations:", max_external_iterations)
print("how many maximum gradient are selected", n_max_grads)
print("The optimizer method used:", optimizer)
print("Tolerance for reaching convergence", tolerance)



curr_state = hf_state
curr_state_open_f = 1.0 * reference_ket
gamma_0 = 11
curr_state_open_g = gamma_0 * reference_ket
gamma_1 = 22
curr_state_open_h = gamma_1 * reference_ket
gamma_2 = 23
curr_state_open_k = gamma_2 * reference_ket


prev_norm = 0.0



for n_iter in range(0, max_external_iterations):
    print("\n\n\n")
    print(
        " --------------------------------------------------------------------------"
    )
    print("                     Fermionic_ADAPT-VQD iteration: ", n_iter)
    print(
        " --------------------------------------------------------------------------"
    )
    # few commands to initialize at each new iteration
    list_grad_ovl_3, curr_norm_ovl_3, next_deriv_ovl_3, next_index_ovl_3 = return_gradient_overlap_1(
        cluster_ops_sparse, hamiltonian_sparse, curr_state_open_f, curr_state_open_g, curr_state_open_h, curr_state_open_k, gamma_0, gamma_1, gamma_2
        )


    # the norm from the current iteration:
    curr_norm_ovl_3 = np.sqrt(curr_norm_ovl_3)
    print(" Norm of the gradients in current iteration = %12.8f" % curr_norm_ovl_3)
    print(" Max gradient in current iteration= %12.8f" % next_deriv_ovl_3)
    print(" Index of the Max gradient in current iteration= ", next_index_ovl_3)
    
    converged = False
    if type_conver == "norm":
        if curr_norm_ovl_3 < threshold_needed:
            converged = True
    else:
        print(" type convergence is not defined")
        exit()
    if converged or (abs(curr_norm_ovl_3 - prev_norm) < 10 ** (-8)):
        print("Convergence is done")
        result["Number_operators"] = len(ansatz_ops)
        result["final_norm"] = curr_norm_ovl_3
        result["parameters"] = parameters_ansatz
        print(" -----------Final ansatz----------- ")
        print(" *final converged energy iteration is %20.12f" % Optimizer_2.fun)
        result["final_energy_last_iteration"] = Optimizer_2.fun
        break

    ansatz_ops = cluster_ops_sp
    ansatz_mat =  cluster_ops_sparse


    circuits = circuits_store
    
    input_funct, callback = opt_funct(circuits, model, qpu, nqbits, energy_lists, beta_1, beta_2)
    options = {"disp": True, "maxiter": 3000, "gtol": 1e-7}
    Optimizer_2 = scipy.optimize.minimize(input_funct, x0=theta_0, method=method, callback=callback, options=options)
    xlist_ovl_2 = Optimizer_1.x
    
    
    print(" Finished energy iteration_i: %20.12f" % Optimizer_2.fun)
    print(" -----------New ansatz created----------- ")
    print(" %4s \t%s \t%s" % ("#", "Coefficients", "Term"))
    parameters_ansatz = []

    for si in range(len(cluster_ops_sp)):
        print(" %4i \t%f " % (si, xlist_ovl_2[si]))
        parameters_ansatz.append(xlist_ovl_2[si])

    curr_state = prepare_state_ansatz(ansatz_ops, hf_init_sp, parameters_ansatz) 
    curr_state_open_f = prepare_adapt_state(reference_ket, ansatz_mat, parameters_ansatz) # this is for the current state of the HF
    curr_state_open_g = prepare_adapt_state(stv_ground_mat_sparse, ansatz_mat, parameters_ansatz)
    curr_state_open_h = prepare_adapt_state(stv_exci_1_mat_sparse, ansatz_mat, parameters_ansatz)
    curr_state_open_k = prepare_adapt_state(stv_exci_2_mat_sparse, ansatz_mat, parameters_ansatz)
    

    prev_norm = curr_norm_g